Допустим вы долго-долго обучали свою модель и хотите уметь её использовать впоследствии без необходимости переучиывать её сначала по полчаса. Для этого нужно уметь сохранять веса модели и загружать их.

In [ ]:
## эта ячейка - не для запуска, и не будет работать если пробовать её запустить!

# метод state_dict() извлекает обученные веса из модели, а torch.save - сохраняет их в путь path
torch.save(model.state_dict(), path)

# загрузка весов модели

model = TheModelClass() # сперва нужно обязательно заново инициализировать ваш класс модели
model.load_state_dict(torch.load(path)) # загружаем в инициализированный строкой ранее класс модели
                                        # веса, лежащие в файле с путем path (тот же самый, что мы использовали при сохранении)

Попробуйте обучить модель со вчера (здесь я скопировала код из ноутбука day3_mnist_conv), сохранить её у себя на гугл диске, а потом перезапустить среду ноутбука, загрузить модель и увидеть, что она работает точно также, как после обучения (без необходимости обучать заново).

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import os

In [2]:
num_epochs = 5
batch_size = 100
learning_rate = 0.001

train_dataset  = dsets.MNIST(root='./data/',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

test_dataset = dsets.MNIST(root='./data/',
                          train=False,
                          transform=transforms.ToTensor())

train_loader  = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2), #1 * 28 * 28 -> 16 * 28 * 28
            nn.BatchNorm2d(16), #16 * 28 * 28
            nn.ReLU(),                 #16 * 28 * 28
            nn.MaxPool2d(2))       # 16 * 14 * 14
        self.layer2  = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2), # 16 * 14 * 14 -> 32 * 14 * 14
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)) # 32 * 7 * 7
        self.fc = nn.Linear(32 * 7 * 7, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
    
cnn = CNN()

In [4]:
criterion = nn.CrossEntropyLoss() # Loss
optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        
        optimizer.zero_grad()
        outputs = cnn(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], lter [%d/%d] Loss: %.4f'
                  %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data))

Epoch [1/5], lter [100/600] Loss: 0.1954
Epoch [1/5], lter [200/600] Loss: 0.1301
Epoch [1/5], lter [300/600] Loss: 0.0534
Epoch [1/5], lter [400/600] Loss: 0.0832
Epoch [1/5], lter [500/600] Loss: 0.0476
Epoch [1/5], lter [600/600] Loss: 0.0538
Epoch [2/5], lter [100/600] Loss: 0.0840
Epoch [2/5], lter [200/600] Loss: 0.0329
Epoch [2/5], lter [300/600] Loss: 0.0153
Epoch [2/5], lter [400/600] Loss: 0.0225
Epoch [2/5], lter [500/600] Loss: 0.0704
Epoch [2/5], lter [600/600] Loss: 0.0127
Epoch [3/5], lter [100/600] Loss: 0.0231
Epoch [3/5], lter [200/600] Loss: 0.1083
Epoch [3/5], lter [300/600] Loss: 0.0376
Epoch [3/5], lter [400/600] Loss: 0.0100
Epoch [3/5], lter [500/600] Loss: 0.0242
Epoch [3/5], lter [600/600] Loss: 0.0501
Epoch [4/5], lter [100/600] Loss: 0.0517
Epoch [4/5], lter [200/600] Loss: 0.0435
Epoch [4/5], lter [300/600] Loss: 0.0066
Epoch [4/5], lter [400/600] Loss: 0.0202
Epoch [4/5], lter [500/600] Loss: 0.0144
Epoch [4/5], lter [600/600] Loss: 0.0250
Epoch [5/5], lte

In [7]:
cnn.eval()
correct = 0
total = 0
for images, labels in test_loader:

    outputs = cnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
print('test Accuracy 10000 test images  = %f %%' % (100 * correct/total))

test Accuracy 10000 test images  = 98.900002 %


В следующих двух ячейках вы сохраняете модель, а после перезапускаете ноутбук

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

path = "/content/gdrive/MyDrive/random_model_name"

In [ ]:
## здесь напишите код для сохранения модели
## как напишите - позовите меня, я проверю что все правильно и что ноутбук можно перезапустить 
## (чтобы не потерять веса и не повторять процесс обучения)

Перезапускаем ноутбук

In [ ]:
## здесь пишем код для загрузки весов в модель
## и не забываем заново инициализировать модель

Проверяем что точность модели не изменилась

In [ ]:
cnn.eval()
correct = 0
total = 0
for images, labels in test_loader:

    outputs = cnn(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()
    
print('test Accuracy 10000 test images  = %f %%' % (100 * correct/total))